In [1]:
import pandas as pd
import numpy as np
#logistics regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report

import matplotlib.pylab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn import linear_model
import scipy.stats as stat

class LogisticReg:
    """
    Wrapper Class for Logistic Regression which has the usual sklearn instance 
    in an attribute self.model, and pvalues, z scores and estimated 
    errors for each coefficient in 
    
    self.z_scores
    self.p_values
    self.sigma_estimates
    
    as well as the negative hessian of the log Likelihood (Fisher information)
    
    self.F_ij
    """
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)
        #### Get p-values for the fitted model ####
        denom = (2.0*(1.0+np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X/denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.array([np.sqrt(Cramer_Rao[i,i]) for i in range(Cramer_Rao.shape[0])]) # sigma for each coefficient
        z_scores = self.model.coef_[0]/sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values
        
        self.z_scores = z_scores
        self.p_values = p_values
        self.sigma_estimates = sigma_estimates
        self.F_ij = F_ij

In [3]:
def summary(X,logit,logit1):
    names = list(X.columns)
    summary_90 = pd.DataFrame(index=names,columns=['Coefficient','p-value','<.10','<.05','<.01'])
    summary_90['Coefficient']=logit1.coef_[0]
    summary_90['p-value']=logit.p_values
    summary_90['Coefficient']=summary_90['Coefficient'].apply(lambda x: float('{:.2e}'.format(x)))
    summary_90['p-value']=summary_90['p-value'].apply(lambda x: float('{:.2e}'.format(x)))
    ten=[]
    five=[]
    one=[]
    for i in range(0,len(X.columns)):
        if summary_90['p-value'][i]<0.1:
            ten.append('*')
        else:
            ten.append('')
        if summary_90['p-value'][i]<0.05:
            five.append('*')
        else:
            five.append('')
        if summary_90['p-value'][i]<0.01:
            one.append('*')
        else:
            one.append('')
    summary_90['<.10']=ten
    summary_90['<.05']=five
    summary_90['<.01']=one
    intercept = float('{:.2e}'.format(logit1.intercept_[0]))
    inter=pd.DataFrame({'Coefficient': [intercept],'p-value':'','<.10':'','<.05':'','<.01':''},index=['Intercept'])
    summary_90 = pd.concat([inter,summary_90])
    return summary_90

In [4]:
data = pd.read_csv('UDPNY_gerardo.csv')

In [5]:
data['all_li_90'] = data['vli_90']+data['li_90']
data['all_li_00'] = data['vli_00']+data['li_00']
data['all_li_16'] = data['vli_16']+data['li_16']

In [6]:
data_aram = pd.read_csv('final_typology9.csv')

In [8]:
len(data_aram)

5374

In [7]:
data = data.rename(columns={'Unnamed: 0':'geoid2'})

In [53]:
data = data.merge(data_aram[['geoid2','typology_2']], on='geoid2')

In [54]:
data = data.dropna(how='any', axis=0)

In [56]:
data = data.reset_index().drop(columns='index')

l=[]
for i in range(0,len(data)):
    if ((data.typology_2[i]=='LI - Ongoing Gentrification') | 
        (data.typology_2[i]=='MHI - Exclusion')):
        l.append(1)
    else :
        l.append(0)

l=[]
for i in range(0,len(data)):
    if ((data.typology_2[i]=='MHI - Exclusion')|(data.typology_2[i]=='MHI - Exclusion')):
        l.append(1)
    else :
        l.append(0)

In [57]:
l=[]
for i in range(0,len(data)):
    if ((data.typology_2[i]=='LI - Ongoing Gentrification and/or Displacement') | 
        (data.typology_2[i]=='MHI - Advanced Gentrification')):
        l.append(1)
    else :
        l.append(0)

In [58]:
data['gen'] = l

In [59]:
data['gen'].sum()

192

In [60]:
data['TOD_pre_00'] = data['TOD_pre_00'].apply(lambda x: float(x))
data['TOD_00_10'] = data['TOD_00_10'].apply(lambda x: float(x))
data['TOD_10'] = data['TOD_10'].apply(lambda x: float(x))
data['TOD'] = data['TOD'].apply(lambda x: float(x))

In [61]:
logit_90 = LogisticReg(C = 10000)
logit1_90 = LogisticRegression(C = 10000)
logit_00 = LogisticReg(C = 10000)
logit1_00 = LogisticRegression(C = 10000)
logit_16 = LogisticReg(C = 10000)
logit1_16 = LogisticRegression(C = 10000)
logit = LogisticReg(C = 10000)
logit1 = LogisticRegression(C = 10000)

In [62]:
data['hinc_90_norm'] = data['hinc_90']/np.max(data['hinc_90'])
data['hinc_00_norm'] = data['hinc_00']/np.max(data['hinc_00'])
data['hinc_16_norm'] = data['hinc_16']/np.max(data['hinc_16'])
data['empd02_norm'] = data['empd02']/np.max(data['empd02'])
data['empd15_norm'] = data['empd15']/np.max(data['empd15'])

In [63]:
data['downtown'] = data['downtown'].apply(lambda x: int(x))

In [64]:
X_90 = data[['hinc_90_norm', 'per_rent_90', 'per_black_90', 'per_asian_90', 'per_latino_90', 'per_hhwchild_90',
             'per_commute_low_90', 'per_commute_med_90', 'per_commute_high_90', 'per_built_80_90',
             'TOD_pre_00', 'per_units_pre50','per_units_post80', 'per_car_commute_90', 'downtown',
            'empd02_norm']]

X_00 = data[['hinc_00_norm', 'per_rent_00', 'per_black_00', 'per_asian_00', 'per_latino_00', 'per_hhwchild_00',
             'per_commute_low_00', 'per_commute_med_00', 'per_commute_high_00', 'per_built_90_00',
             'TOD_pre_00', 'per_units_pre50_16', 'per_burden_00','downtown','empd02_norm', 'per_units_pre50_00',
            'per_car_commute_00']]

X_16 = data[['hinc_16_norm', 'per_rent_16','per_asian_16', 'per_black_16',
             'per_hhwchild_16', 'per_commute_low_16', 'per_commute_med_16', 'per_commute_high_16', 'per_built_00_16',
             'TOD', 'per_units_pre50_16', 'per_latino_16', 'per_burden_16','empd15_norm']]

In [65]:
X_90 = X_90.reset_index().drop('index',axis=1)
X_00 = X_00.reset_index().drop('index',axis=1)
X_16 = X_16.reset_index().drop('index',axis=1)

In [66]:
Y_90_00 = data.gen

In [67]:
Y_16 = data.gen

In [68]:
Y_90_00 = Y_90_00.reset_index().drop('index',axis=1)

In [69]:
Y_16 = Y_16.reset_index().drop('index',axis=1)

In [70]:
len(Y_16)

3681

In [71]:
len(Y_90_00)

3681

In [72]:
logit_90.fit(X_90, Y_16.values.ravel())
logit1_90.fit(X_90, Y_16.values.ravel())

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [73]:
logit_00.fit(X_00, Y_90_00.values.ravel())
logit1_00.fit(X_00, Y_90_00.values.ravel())

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [74]:
logit_16.fit(X_16, Y_16.values.ravel())
logit1_16.fit(X_16, Y_16.values.ravel())

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [75]:
summary_90 = summary(X_90,logit_90,logit1_90)
summary_00 = summary(X_00,logit_00,logit1_00)
summary_16 = summary(X_16,logit_16,logit1_16)

In [76]:
summary_90

,Coefficient,p-value,<.10,<.05,<.01
Intercept,2.8900,,,,
hinc_90_norm,-8.1600,8.74e-06,*,*,*
per_rent_90,-1.7000,0.0198,*,*,
per_black_90,-2.1900,0.00409,*,*,*
per_asian_90,-4.4000,0.00168,*,*,*
per_latino_90,-2.1700,0.0109,*,*,
per_hhwchild_90,-3.0600,2.82e-13,*,*,*
per_commute_low_90,-0.1130,0.936,,,
per_commute_med_90,2.0100,0.139,,,
per_commute_high_90,0.9960,0.511,,,


In [77]:
summary_00

,Coefficient,p-value,<.10,<.05,<.01
Intercept,-1.8000,,,,
hinc_00_norm,0.6960,0.639,,,
per_rent_00,0.8940,0.216,,,
per_black_00,0.5950,0.153,,,
per_asian_00,0.3870,0.655,,,
per_latino_00,1.6200,0.00748,*,*,*
per_hhwchild_00,-6.6100,5.99e-08,*,*,*
per_commute_low_00,-2.3700,0.0683,*,,
per_commute_med_00,0.1540,0.9,,,
per_commute_high_00,0.4130,0.747,,,


In [78]:
summary_16

,Coefficient,p-value,<.10,<.05,<.01
Intercept,-3.4500,,,,
hinc_16_norm,4.0900,0.000165,*,*,*
per_rent_16,2.3000,0.000104,*,*,*
per_asian_16,1.2900,0.0651,*,,
per_black_16,0.6470,0.165,,,
per_hhwchild_16,-2.9000,9.64e-08,*,*,*
per_commute_low_16,-4.4100,1.96e-06,*,*,*
per_commute_med_16,-0.2300,0.814,,,
per_commute_high_16,1.1900,0.259,,,
per_built_00_16,4.3800,3.84e-08,*,*,*


In [ ]:
pred_val_90 = logit1_90.predict(X_90)
pred_val_00 = logit1_00.predict(X_00)
pred_val_16 = logit1_16.predict(X_16)

In [ ]:
print(accuracy_score(Y_90_00, pred_val_90),
accuracy_score(Y_90_00, pred_val_00),
accuracy_score(Y_16, pred_val_16))